# Importing the libraries

In [1]:
from langchain.llms.openai import OpenAIChat
from langchain.prompts import PromptTemplate
from langchain.output_parsers import (
    ResponseSchema,
    StructuredOutputParser,
    PydanticOutputParser,
)
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.chains import LLMChain
from langchain.globals import set_debug
from dotenv import load_dotenv

set_debug(True)  # Enable debug mode
load_dotenv()  # Load the variables from .env file

True

# Models

In this example we will use OpenAI's LLM Models to generate text, But you can use any LLM you want from the supported llms within langchain e.i. HuggingFace, LLaMa, PaLM, and etc.





Specifically we will use the newer models that are connected to `v1/chat/completion` endpoint, and can be used in `OpenAIChat` class. 

for more information about the models and the endpoint please refer to [OpenAI's API Docs](https://platform.openai.com/docs/guides/text-generation/chat-completions-vs-completions)

To setup your OpenAI API key: 
1. Create an OpenAI account
2. Create a new API key from this [link](https://platform.openai.com/api-keys)
3. Copy the API key and paste it in the `OPENAI_API_KEY` variable in `.env` file



In [2]:
# Temperature: a number between 0 and 1. The closer to 0,\
# the more predictable the text is, the closer to 1, the more Hallucinations.

llm = OpenAIChat(
    model="gpt-3.5-turbo-1106",
    temperature=0.0,
)

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [3]:
# To make sure the model is working, we can test it with a prompt.

response = llm("Hello, how are you?")

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Hello, how are you?"
  ]
}


[llm/end] [1:llm:OpenAIChat] [2.63s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello! I'm just a language model AI, so I don't have feelings, but I'm here to help you. How can I assist you today?",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 13,
      "completion_tokens": 32,
      "total_tokens": 45
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}


In [4]:
response

"Hello! I'm just a language model AI, so I don't have feelings, but I'm here to help you. How can I assist you today?"

# Prompts

When building apps you need the model to generate output in specific format each inference, so when building a prompt you need to follow some prompt engineering techniques, which are discussed in [Deeplearning.ai's ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/chatgpt-prompt-eng) go check it out it's free.

I've created prompts that meets our needs in the examples that follows the guidelines provided in the short course, so don't worry about it being complicated.

In [5]:
text = """
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

In [6]:
# We will use f-strings to insert the text into the prompt.
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

In [7]:
print(prompt)


Summarize the text delimited by triple backticks \ 
into a single sentence.
```
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
```



In [8]:
# Getting the response from the model.
response = llm(prompt)

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Summarize the text delimited by triple backticks \\ \ninto a single sentence.\n```\nYou should express what you want a model to do by \\ \nproviding instructions that are as clear and \\ \nspecific as you can possibly make them. \\ \nThis will guide the model towards the desired output, \\ \nand reduce the chances of receiving irrelevant \\ \nor incorrect responses. Don't confuse writing a \\ \nclear prompt with writing a short prompt. \\ \nIn many cases, longer prompts provide more clarity \\ \nand context for the model, which can lead to \\ \nmore detailed and relevant outputs.\n```"
  ]
}


[llm/end] [1:llm:OpenAIChat] [4.55s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Clear and specific instructions for a model will guide it towards the desired output and reduce the chances of irrelevant or incorrect responses, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 134,
      "completion_tokens": 45,
      "total_tokens": 179
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}


Prompts sometimes will be long and contains lots of parameters, so Langchain provides a simple way to generate prompts for your LLMs, you can use the `PromptTemplate` class to generate prompts for your LLMs.

In [9]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text delimited by triple backticks, \
please extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python List.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

and the values for each key are the information extracted from the text.

```{text}```
"""

In [10]:
prompt_template = PromptTemplate(
    input_variables=["text"],
    template=review_template,
)

In [11]:
prompt = prompt_template.format(text=customer_review)

In [12]:
print(prompt)

For the following text delimited by triple backticks, please extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python List.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

and the values for each key are the information extracted from the text.

```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out 

In [13]:
response = llm(prompt)

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "For the following text delimited by triple backticks, please extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python List.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\nand the values for each key are the information extracted from the text.\n\n```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morni

[llm/end] [1:llm:OpenAIChat] [2.96s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\n  \"gift\": true,\n  \"delivery_days\": 2,\n  \"price_value\": [\"It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\"]\n}",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 239,
      "completion_tokens": 48,
      "total_tokens": 287
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}


In [14]:
print(response)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


# Parsers

The output response from the LLMs is not always in the format you want. So Langchain provides a simple way to parse the output response from the LLMs, you can use different Parser classes to parse the output response based on your desired format.

In [15]:
type(response)

str

The type of the response without using any parser is a `str`, so we will use a parser to parse the response to the desired format.

In [16]:
parser = SimpleJsonOutputParser()

In [17]:
dict_resp = parser.parse(response)
dict_resp

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [18]:
type(dict_resp)

dict

There are more sophisticated parsers that can add Schema, and Validation to the response, e.i. `StructuredOutputParser` and `PydanticOutputParser`.

## StructuredOutputParser

In [19]:
# Define the schema for the output.
response_schemas = [
    ResponseSchema(
        name="gift",
        type="bool",
        description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown",
    ),
    ResponseSchema(
        name="delivery_days",
        type="int",
        description="How many days did it take for the product to arrive? If this information is not found, output -1.",
    ),
    ResponseSchema(
        name="price_value",
        type="list",
        description="Extract any sentences about the value or price, and output them as a comma separated Python List.",
    ),
]

In [20]:
structured_parser = StructuredOutputParser(response_schemas=response_schemas)

When using `StructuredOutputParser` and providing a `response_schema`, we can use the response schema in the prompt template to generate the prompt with the correct format, and the parser will parse the response to the provided schema.

In [21]:
# Instead of hardcoding the output schema, we can use different parsers to add it automatically.
structured_review_template = """\
The customer review for a product is delimited by triple backticks.
```{text}```

Extract the following information from the review, \
and output it in the specified format:
{format_instructions}
"""

In [22]:
structured_prompt_template = PromptTemplate(
    template=structured_review_template,
    input_variables=["text"],
    partial_variables={
        "format_instructions": structured_parser.get_format_instructions()
    },
)

In [23]:
structured_prompt = structured_prompt_template.format(text=customer_review)
print(structured_prompt)

The customer review for a product is delimited by triple backticks.
```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
```

Extract the following information from the review, and output it in the specified format:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": bool  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown
	"delivery_days": int  // How many days did it take for the product to arrive? If this information is 

In [24]:
structured_response = llm(structured_prompt)

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "The customer review for a product is delimited by triple backticks.\n```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\n```\n\nExtract the following information from the review, and output it in the specified format:\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing \"```json\" and \"```\":\n\n```json\n{\n\t\"gift\": bool  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown\n\t\

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-1106 in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-1106 in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-1106 in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


[llm/end] [1:llm:OpenAIChat] [46.91s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n\t\"gift\": true,\n\t\"delivery_days\": 2,\n\t\"price_value\": [\"It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\"]\n}\n```",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 270,
      "completion_tokens": 52,
      "total_tokens": 322
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}


In [25]:
print(structured_response)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [26]:
structured_parser.parse(structured_response)

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

## PydanticOutputParser


In [27]:
class Review(BaseModel):
    gift: bool = Field(
        description="Was the item purchased as a gift for someone else?, True if yes, False if not or unknown.",
        default=False,
    )
    delivery_days: int = Field(
        description="How many days did it take for the product to arrive? If this information is not found, output -1.",
        default=-1,
        gt=-1,
    )
    price_value: List[str] = Field(
        description="Extract any sentences about the value or price, and output them as a comma separated Python List.",
        default=[],
    )

In [28]:
pydantic_parser = PydanticOutputParser(pydantic_object=Review)

In [29]:
pydantic_prompt_template = PromptTemplate(
    template=structured_review_template,
    input_variables=["text"],
    partial_variables={
        "format_instructions": pydantic_parser.get_format_instructions()
    },
)

In [30]:
pydantic_prompt = pydantic_prompt_template.format(text=customer_review)
print(pydantic_prompt)

The customer review for a product is delimited by triple backticks.
```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
```

Extract the following information from the review, and output it in the specified format:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"prop

In [31]:
pydantic_response = llm(pydantic_prompt)

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "The customer review for a product is delimited by triple backticks.\n```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\n```\n\nExtract the following information from the review, and output it in the specified format:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"requ

[llm/end] [1:llm:OpenAIChat] [3.25s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\n  \"gift\": true,\n  \"delivery_days\": 2,\n  \"price_value\": [\"It arrived in two days, just in time for my wife's anniversary present.\", \"It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\"]\n}",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 427,
      "completion_tokens": 65,
      "total_tokens": 492
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}


In [32]:
print(pydantic_response)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It arrived in two days, just in time for my wife's anniversary present.", "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [33]:
# Returns a Review object.
pydantic_parser.parse(pydantic_response)

Review(gift=True, delivery_days=2, price_value=["It arrived in two days, just in time for my wife's anniversary present.", "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."])

# Chain

We can connect the Prompt Template, Model, and the Parser together in one object called a `Chain`, and then use it to generate output.

In [34]:
structured_chain = LLMChain(
    llm=llm,
    prompt=structured_prompt_template,
    output_parser=structured_parser,
)

In [35]:
structured_chain({"text": customer_review})

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "text": "This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\n"
}
[llm/start] [1:chain:LLMChain > 2:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "The customer review for a product is delimited by triple backticks.\n```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using

[llm/end] [1:chain:LLMChain > 2:llm:OpenAIChat] [2.71s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n\t\"gift\": true,\n\t\"delivery_days\": 2,\n\t\"price_value\": [\"It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\"]\n}\n```",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 270,
      "completion_tokens": 52,
      "total_tokens": 322
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:LLMChain] [2.71s] Exiting Chain run with output:
{
  "text": {
    "gift": true,
    "delivery_days": 2,
    "price_value": [
      "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
    ]
  }
}


{'text': {'gift': True,
  'delivery_days': 2,
  'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}}

In [36]:
pydantic_chain = LLMChain(
    llm=llm,
    prompt=pydantic_prompt_template,
    output_parser=pydantic_parser,
)

In [37]:
pydantic_chain({"text": customer_review})

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "text": "This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\n"
}
[llm/start] [1:chain:LLMChain > 2:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "The customer review for a product is delimited by triple backticks.\n```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-1106 in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-1106 in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langcha

[llm/end] [1:chain:LLMChain > 2:llm:OpenAIChat] [24.39s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\n  \"gift\": false,\n  \"delivery_days\": 2,\n  \"price_value\": [\"It arrived in two days, just in time for my wife's anniversary present.\", \"It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\"]\n}",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 427,
      "completion_tokens": 65,
      "total_tokens": 492
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:LLMChain] [24.40s] Exiting Chain run with output:
[outputs]


{'text': Review(gift=False, delivery_days=2, price_value=["It arrived in two days, just in time for my wife's anniversary present.", "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."])}